In [ ]:
import pandas as pd
from edisgo.edisgo import import_edisgo_from_files
from edisgo.flex_opt.costs import grid_expansion_costs
from edisgo.tools.tools import reduce_timeseries_data_to_given_timeindex
from copy import deepcopy
from edisgo.tools.config import Config
import os

In [ ]:
grid = "5bus_testcase"
path = os.path.join(".", "data", grid)
solution_path = os.path.join(".","opf_solutions", grid+".csv")

In [ ]:
edisgo = import_edisgo_from_files(path, 
                                  import_topology=True,
                                  import_timeseries=True,
                                  import_electromobility=True,
                                  import_heat_pump=True,
                                  import_dsm=True,
                                  import_overlying_grid=True)
edisgo._config =  Config()
edisgo_OPF = deepcopy(edisgo)
edisgo_OPF.timeseries.timeindex.freq = "1H"
edisgo.reinforce()
cost = grid_expansion_costs(edisgo)

# 1. Flexibilitäten
Implementiert sind folgende Flexibilitäten
* Elektromobilität
* Wärmepumpen mit Wärmespeichern
* Batteriespeicher
* Demand Side Management

Damit die flexiblen Verbraucher in der Julia-Optimierung berücksichtigt werden, müssen für jede Flexibilität eine Liste der Namen der flexiblen Verbraucher an den entsprechenden Parameter in ```edisgo.opf.powermodels_opf.pm_optimize()``` übergeben. 

Parameter:
* flexible_cps
* flexible_hps
* flexible_storage_units
* flexible_loads

Falls Flexibilität nicht mit optimiert werden soll, den Parameter nicht setzen. 

In [ ]:
# Elektromobilität
flexible_cps = edisgo.topology.loads_df.loc[(edisgo.topology.loads_df.type=="charging_point") & 
                                            ((edisgo.topology.loads_df.sector=="home")|
                                             (edisgo.topology.loads_df.sector=="work"))].index.values
# Wärmepumpen/-speicher
flexible_hps = edisgo.heat_pump.thermal_storage_units_df.index.values
# Batteriespeicher
flexible_storage_units = edisgo.topology.storage_units_df.index.values
# DSM
flexible_loads = edisgo.dsm.p_max.columns

# 2. OPF Version
Parameter: opf_version
1. Ohne Anforderungen aus dem übergelagerten Netz, ohne Netzrestriktionen
2. Ohne Anforderungen aus dem übergelagerten Netz, mit Netzrestriktionen
3. Mit Anforderungen aus dem übergelagerten Netz, ohne Netzrestriktionen
4. Mit Anforderungen aus dem übergelagerten Netz, mit Netzrestriktionen

In [ ]:
opf_version = 2

# 3. Optimierungsmethode
Parameter: method
- "soc": konvexe SOC-Optimierung
- "nc": nicht-konvexe Optimierung

In [ ]:
method = "soc"

# 4. eDisGo Funktionen für die Optimierung
## 4.1. ```to_powermodels()```
Wandelt eDisGo Objekt in PowerModels Network Data Format um.  
```python
to_powermodels(
    self,
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
    )
```

In [ ]:
pm, hv_flex_dict = edisgo_OPF.to_powermodels(flexible_cps=flexible_cps, flexible_hps=flexible_hps,
                               flexible_loads=flexible_loads, flexible_storage_units=flexible_storage_units,
                               opf_version=opf_version)
pm

## 4.3. ```save_edisgo_to_json()```
Speichert eDisGo Objekt im PowerModels Network Data Format als json. Wird zum Debuggen mit Julia benötigt.
```python
save_edisgo_to_json(
    self,
    filename=None,
    path="",
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
)

```

## 4.4. ```pm_optimize()```
Startet die Julia Optimierung für das eDisGo Objekt in einem Subproccess und schreibt die Ergebnisse der Optimierung wieder auf das eDisGo Objekt.
```python
pm_optimize(
    self,
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
    method="soc",
    warm_start=False,
    silence_moi=False,
    save_heat_storage=False,
    save_slack_gen=False,
    save_slacks=False,
    path="",
)

```

In [ ]:
edisgo_OPF.pm_optimize(flexible_cps=flexible_cps,
                       flexible_hps=flexible_hps,
                       flexible_loads=flexible_loads,
                       flexible_storage_units=flexible_storage_units,
                       s_base=1,
                       opf_version=opf_version, 
                       silence_moi=False)
edisgo_OPF.opf_results.solution_time

## 4.2. ```from_powermodels()```
Schreibt Ergebnisse der PowerModels Optimierung (Zeitreihen für die Flexibilitäten) zurück auf das eDisGo Objekt. Wird in Funktion ```pm_optimize()``` beutzt. 
```python
from_powermodels(
    self,
    pm_results,
    hv_flex_dict,
    s_base=1,
    save_heat_storage=False,
    save_slack_gen=False,
    save_slacks=False,
    path="",
)
```

In [ ]:
edisgo_OPF.reinforce()
cost_OPF = grid_expansion_costs(edisgo_OPF)

In [ ]:
cost

In [ ]:
cost_OPF